In [1]:
import pandas as pd
import numpy as np
import re

from keras.layers import Embedding, Dense, LSTM, Dense, Input, concatenate
from keras.models import Model
from keras.utils import plot_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
df = pd.read_csv('new_labeled_tweets1.csv', nrows=1500000)
#df = df['text'].str.split(' ')
#to_remove = splits.apply(lambda x: len(x)).sort_values(ascending=False)[:10].index
#df = df.drop(to_remove).reset_index(drop=True)
#print(splits)

In [3]:
df = df.dropna(subset=['text'])
splits = df['text'].str.split(' ')
to_remove = splits.apply(lambda x: len(x)).sort_values(ascending=False)[:10].index
df = df.drop(to_remove).reset_index(drop=True)

In [39]:
#df = df.dropna(subset=['text'])
#df = df.drop_duplicates('text', inplace=True)
#df = df.fillna("")
#df.dropna(axis=1, how='any')
#df = df.dropna(thresh=1)
#del df['$url$']
#df1 = df.loc[df.notna().all()]
#df = df.drop(df1)
#splits= dftext'].str.split(' ')
#to_remove = splits.apply(lambda x: len(x)).sort_values(ascending=False)[:10].index
#df = df.drop(to_remove).reset_index(drop=True)

In [4]:
df['text'] = df['text'].str.replace('<quoted_status>', '<quoted_status> ')
df['text'] = df['text'].str.replace('<hashtag>', '<hashtag> ')
df['text'] = df['text'].apply(lambda x: re.sub( '\s+', ' ', x ).strip())

In [5]:
df_clean_split = df['text'].str.split(' ', expand=True)
words = df_clean_split.stack().unique()
max_sequence = df_clean_split.shape[1]

In [6]:
tockenizer = Tokenizer(words.shape[0]) 
tockenizer.fit_on_texts(df['text'])
sequences = tockenizer.texts_to_sequences(df['text'])

In [7]:
word_index = tockenizer.word_index
data = pad_sequences(sequences, maxlen=max_sequence)

In [8]:
embeddings_index = {}
f = open('glove.twitter.27B.50d.txt', encoding="utf8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [9]:
embedding_matrix = np.zeros((len(word_index) + 1, 50))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [10]:
X_text_train = data[:1100000,:]
X_text_test = data[1100000:,:]

In [11]:
number_data = df[['retweet_count', 'favorite_count', 'reply_count', 'hashtag_count', 'mention_count', 'url_count']].values

X_number_train = number_data[:1100000,:]
X_number_test = number_data[1100000:,:]

In [12]:
y_vals = df['BotOrNot'].values

y_train = y_vals[:1100000].reshape(-1,1)
y_test = y_vals[1100000:].reshape(-1,1)

In [13]:
embedding_layer = Embedding(len(word_index) + 1,
                            50,
                            weights=[embedding_matrix],
                            input_length=max_sequence,
                            trainable=False)

In [14]:
main_input = Input(shape=(max_sequence,), dtype='int32', name='main_input')
embedded_sequences = embedding_layer(main_input)

lstm_out = LSTM(32)(embedded_sequences)

auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

auxiliary_input = Input(shape=(6,), name='aux_input')

x = concatenate([lstm_out, auxiliary_input])

x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)

main_output = Dense(1, activation='sigmoid', name='main_output')(x)

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

In [16]:
model.compile(optimizer='adam',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2},
              metrics=['accuracy'])

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 318)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 318, 50)      21998100    main_input[0][0]                 
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           10624       embedding_1[0][0]                
__________________________________________________________________________________________________
aux_input (InputLayer)          (None, 6)            0                                            
__________________________________________________________________________________________________
concatenat

In [18]:
model.fit({'main_input': X_text_train, 'aux_input': X_number_train},
          {'main_output': y_train, 'aux_output': y_train},
          validation_data=[{'main_input': X_text_test, 'aux_input': X_number_test}, {'main_output': y_test, 'aux_output': y_test}],
          epochs=50, 
          batch_size=512)

Instructions for updating:
Use tf.cast instead.
Train on 1100000 samples, validate on 899357 samples
Epoch 1/50
1100000/1100000 [==============================] - 866s 788us/step - loss: 0.2747 - main_output_loss: 0.2262 - aux_output_loss: 0.2426 - main_output_acc: 0.9206 - aux_output_acc: 0.8996 - val_loss: 0.2205 - val_main_output_loss: 0.1801 - val_aux_output_loss: 0.2021 - val_main_output_acc: 0.9380 - val_aux_output_acc: 0.9207
Epoch 2/50
1100000/1100000 [==============================] - 868s 789us/step - loss: 0.2190 - main_output_loss: 0.1811 - aux_output_loss: 0.1898 - main_output_acc: 0.9404 - aux_output_acc: 0.9273 - val_loss: 0.2080 - val_main_output_loss: 0.1721 - val_aux_output_loss: 0.1795 - val_main_output_acc: 0.9429 - val_aux_output_acc: 0.9329
Epoch 3/50
1100000/1100000 [==============================] - 876s 796us/step - loss: 0.2039 - main_output_loss: 0.1691 - aux_output_loss: 0.1742 - main_output_acc: 0.9445 - aux_output_acc: 0.9360 - val_loss: 0.2326 - val_main_

1100000/1100000 [==============================] - 862s 784us/step - loss: 0.1473 - main_output_loss: 0.1203 - aux_output_loss: 0.1349 - main_output_acc: 0.9592 - aux_output_acc: 0.9508 - val_loss: 0.1623 - val_main_output_loss: 0.1334 - val_aux_output_loss: 0.1445 - val_main_output_acc: 0.9548 - val_aux_output_acc: 0.9478
Epoch 49/50
1100000/1100000 [==============================] - 856s 778us/step - loss: 0.1446 - main_output_loss: 0.1176 - aux_output_loss: 0.1350 - main_output_acc: 0.9593 - aux_output_acc: 0.9508 - val_loss: 0.1644 - val_main_output_loss: 0.1355 - val_aux_output_loss: 0.1448 - val_main_output_acc: 0.9542 - val_aux_output_acc: 0.9470
Epoch 50/50
1100000/1100000 [==============================] - 856s 778us/step - loss: 0.1422 - main_output_loss: 0.1154 - aux_output_loss: 0.1345 - main_output_acc: 0.9594 - aux_output_acc: 0.9509 - val_loss: 0.1706 - val_main_output_loss: 0.1418 - val_aux_output_loss: 0.1442 - val_main_output_acc: 0.9549 - val_aux_output_acc: 0.9476


In [36]:
np.array(X_text_train)
predicted_classes = model.predict(X_text_train)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[     0,      0,      0, ...,   7505,    458,    955],
       [     0,      0,      0, ...,     28,      3,   1247],
       [     0,      0,      0, ...,    337,    136,    141],
       ...,
 ...

In [ ]:
from sklearn.metrics import classification_report
predicted_classes = model.predict(X_text_test, batch_size=32, verbose=1 )
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y-test, predicted_classes, target_names=target_names))